In [14]:
import pandas as pd
import numpy as np

In [12]:
data = pd.read_csv("data/202120_Laboratorio_2_datos_BancAlpes_credit_card_data.csv",sep=";", index_col="Id")
data

,Customer,Limit_bal,Sex,Education,Marriage,Age,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
Id,,,,,,,,,,
1,87073,20000,Female,2,Married,24,2,1,1,0
2,38414,120000,Female,2,Single,26,3,0,10,9
3,17341,90000,Female,2,Single,34,7,1,3,4
4,40496,50000,Female,2,Married,37,5,1,1,?
5,47437,50000,Male,2,Married,57,6,0,12,3
...,...,...,...,...,...,...,...,...,...,...
656,51108,280000,Male,2,Married,39,10,1,10,0
657,60732,290000,Female,3,Single,27,10,1,13,2
658,53834,190000,Female,1,Single,28,8,1,9,1


In [15]:
data=data.replace("?",np.nan)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660 entries, 1 to 660
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Customer             660 non-null    object
 1   Limit_bal            660 non-null    object
 2   Sex                  660 non-null    object
 3   Education            660 non-null    object
 4   Marriage             659 non-null    object
 5   Age                  660 non-null    object
 6   Total_Credit_Cards   660 non-null    object
 7   Total_visits_bank    658 non-null    object
 8   Total_visits_online  660 non-null    object
 9   Total_calls_made     659 non-null    object
dtypes: object(10)
memory usage: 56.7+ KB
